In [53]:
using JuMP
using CPLEX
using LinearAlgebra

Now we define the master problem :)

In [54]:
function solve_master(A,b,c, op_A, op_B, feas_A, feas_B)
    nx = size(c, 2)
    m = Model(CPLEX.Optimizer)
    set_optimizer_attribute(m, "CPX_PARAM_SCRIND", 0)
    if size(op_A, 1) == 0
        @variable(m, x[i=1:nx]>=0)
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx))
        @constraint(m, A*x .<= b)
        optimize!(m)
        theta = -Inf
    else
        @variable(m, x[1:nx])
        @variable(m, theta)
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx) + theta)
        @constraint(m, A*x .<= b)
        for i = 1:size(op_A, 1)
            @constraint(m, sum(op_A[i,j] * x[j] for j = 1:nx) + op_A[i,nx+1] * theta .>= op_B[i])
    
        end
    end
    if size(feas_A, 1) != 0
        for i = 1:size(feas_A, 1)
            @constraint(m, sum(feas_A[i,j] * x[j] for j = 1:nx) .>= feas_B[i])
    
        end
    end
    optimize!(m)
    theta = JuMP.value(theta)
    sol = zeros(1, nx)
    for i = 1:nx
        sol[i] = JuMP.value(x[i])
    end
    sol = vec(sol)
    return (objective_value(m), sol, theta, JuMP.termination_status(m))
end

solve_master (generic function with 1 method)

In [55]:
function step2(W, h, T, x, omega, q)
    m = Model(CPLEX.Optimizer)
    ny = size(W,2)
    set_optimizer_attribute(m, "CPX_PARAM_SCRIND", 0)
    @variable(m, y[1:ny] >= 0)
    @constraint(m,constraint, W*y - h[:,omega] + T*x .<= 0)
    @objective(m, Min, transpose(q[omega,:]) * y)
    optimize!(m);
    sol = zeros(1, ny)
    for i = 1:ny
        sol[i] = JuMP.value(y[i])
    end
    sol = vec(sol)
    return (objective_value(m), sol, dual.(constraint), JuMP.termination_status(m))
end

step2 (generic function with 1 method)

In [56]:
function compute_optimality_cuts(W, h, T, x_k, q, p)
    e = 0
    E = zeros(1,size(x_k,1))
    w_now = 0
    y_now = [] 
    for w in 1:size(p,2)
        (w_now, y_now, pi_k, termination_status) = step2(W, h, T, x_k, w, q)
        e += p[w] * transpose(pi_k) * h[:,w]
        E += p[w] * transpose(pi_k) * T
    end
    return(e, E, e - (E*x_k)[1], y_now)
end

compute_optimality_cuts (generic function with 1 method)

In [60]:
function compute_feasibility_cuts(W,h,T,x_k)
    ny = size(W,2)        # number of variable
    nconst = size(W,1)    # number of constraint
    nscen = size(h,2)     # number of scenario
    for k in 1:nscen
        m = Model(CPLEX.Optimizer)
        set_optimizer_attribute(m, "CPX_PARAM_SCRIND", 0)
        @variable(m, y[1:ny] >= 0)
        @variable(m, vplus[1:nconst] >= 0)
        @variable(m, vmoins[1:nconst] >= 0)
        @objective(m, Min, sum(vplus[i] + vmoins[i] for i = 1:nconst))
        @constraint(m, dual_const, W*y + vplus - vmoins .<= h[:,k] - T*x_k) # .<= instead of .== because we add one variable for each equality : x + y = 10 <=> x + y - s <= 10
                                                                            # Dual shows that indead the constraint is tight when w' = 0
        optimize!(m)
        w_prime = objective_value(m)
        dual_val = dual.(dual_const)
        if(w_prime > 0)
            D = transpose(dual_val) * T
            d = transpose(dual_val) * h[:,k]
            return(true,d,D)
        end
    end
    return(false,-1,-1)
end

compute_feasibility_cuts (generic function with 1 method)

In [61]:
function main(A, b, c, T, W, h, q, p)
    op_A = []
    op_B = []
    fea_A = []
    fea_B = []
    k = 0
    val_k, x_k, theta_k = 0, 0, 0
    w_now = -Inf
    y_k = 0
    e = 0
    E = zeros(1,size(p,2))
    while (k<10000000000)
        (val_k, x_k, theta_k, termination_status) = solve_master(A, b, c, op_A, op_B, fea_A, fea_B)
        if termination_status == MOI.INFEASIBLE
            return (-1, -1, -1, -1)
        end
        V_temp = []
        (added_const,d,D) = compute_feasibility_cuts(W,h,T,x_k)
        if !added_const
            (e, E, w_now, y_k) = compute_optimality_cuts(W, h, T, x_k, q, p)
            if(w_now <= theta_k)
                return (x_k, y_k, k+1, val_k)
            end
            V_temp = [E 1]
            if op_A == []
                op_A  = V_temp
                op_B  = [e]    
            else
                op_A = [op_A; V_temp]
                op_B = [op_B;e]
            end
        else
            V_temp = D
            if fea_A == []
                fea_A  = V_temp
                fea_B  = [d]    
            else
                fea_A = [fea_A; V_temp]
                fea_B = [fea_B;d]
            end
        end
        k += 1 
    end
end 

main (generic function with 1 method)

In [62]:

#= Test 1
d = [500 100; 300 300]
q = [-24 -28; -28 -32]
p = [0.4 0.6]

A = [1 1; -1 0; 0 -1]
b = [120; -40; -20]
c = [100 150]

T = [-60 0; 0 -80; 0 0; 0 0]
W = [6 10; 8 5; 1 0; 0 1]
h = [0 0; 0 0; d[1,1] d[2,1]; d[1,2] d[2,2]]
=#
#= Test 2

q = [1 1; 1 1; 1 1]
p = [1/3 1/3 1/3]
A = reshape([1], 1, 1)
b = [10]
c = [0]
T = reshape([1; -1], 2, 1)
W = [1 -1; -1 1]
h = [1 2 4; -1 -2 -4]
=#

# Test 3

q = [15 12; 15 12; 15 12; 15 12]
p = [1/4 1/4 1/4 1/4]
A = reshape([0 0], 1, 2)
b = [1]
c = [3 2]
T = [-1 0; 0 -1; 0 0; 0 0; 0 0; 0 0]
W = [3 2; 2 5; 1 0; 0 1; -1 0; 0 -1]
h = [0 0 0 0; 0 0 0 0; 4 4 6 6; 4 8 4 8; (-0.8*4) (-0.8*4) (-0.8*6) (-0.8*6); (-0.8*4) (-0.8*8) (-0.8*4) (-0.8*8)]

(x,y, k, val) = main(A, b, c, T, W, h, q, p)

println("Solution Found !")
print("\t x = ")
println(x)
print("\t Iterations = ")
println(k)

Solution Found !
	 x = [27.2, 41.6]
	 Iterations = 12
